In [ ]:
!pip install mxnet
!pip install gluonnlp pandas tqdm
!pip install sentencepiece
!pip install transformers
!pip install torch

In [ ]:
!pip install git+https://git@github.com/SKTBrain/KoBERT.git@master

  Cloning https://****@github.com/SKTBrain/KoBERT.git (to revision master) to /tmp/pip-req-build-t4pxxrcr
  Running command git clone -q 'https://****@github.com/SKTBrain/KoBERT.git' /tmp/pip-req-build-t4pxxrcr


In [ ]:
import torch
from torch import nn
import torch.nn.functional as F
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
import gluonnlp as nlp
import numpy as np
from tqdm import tqdm, notebook

import math
#kobert
from kobert.utils import get_tokenizer
from kobert.pytorch_kobert import get_pytorch_kobert_model

#transformers
from transformers import AdamW
from transformers.optimization import get_cosine_schedule_with_warmup

#train & test 데이터로 나누기
from sklearn.model_selection import train_test_split

import pandas as pd

#json 관련 라이브러리
import json

# csv 모듈 import하기
import csv

In [ ]:
#구글 드라이브를 쓰자
from google.colab import drive
#루트를 정해준다.
ROOT = "/content/drive"
print(ROOT)

drive.mount(ROOT) #we mount the google drive at /content/drive

#import join used to join ROOT path and MY_GOOGLE_DRIVE_PATH
from os.path import join


MY_GOOGLE_DRIVE_PATH = '/content/drive/My Drive/Colab Notebooks'

print("MY_GOOGLE_DRIVE_PATH: ", MY_GOOGLE_DRIVE_PATH)

%cd "{MY_GOOGLE_DRIVE_PATH}"

/content/drive
Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
MY_GOOGLE_DRIVE_PATH:  /content/drive/My Drive/Colab Notebooks
/content/drive/My Drive/Colab Notebooks


In [ ]:
import torch
torch.__version__

'1.9.0+cu102'

In [ ]:
import transformers
transformers.__version__

'4.11.2'

In [ ]:
import kobert
kobert.__version__

'0.1.2'

In [ ]:
device = torch.device("cuda:0")

In [ ]:
#BERT 모델, Vocabulary 불러오기
bertmodel, vocab = get_pytorch_kobert_model()

using cached model
using cached model


In [ ]:
# data = pd.read_csv('training_data.csv')
# path = MY_GOOGLE_DRIVE_PATH + "감성대화말뭉치(최종데이터)_Training.xlsx"
# chat_data = pd.read_excel("감성대화말뭉치(최종데이터)_Training.xlsx")
chat_data = pd.read_excel("상황2.xlsx")

In [ ]:
chat_data

,situation,sentence1,sentence2,sentence3,sentence3.1
0,5,아내가 드디어 출산하게 되어서 정말 신이 나.,아 지금 정말 신이 나.,아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.,NaN
1,8,당뇨랑 합병증 때문에 먹어야 할 약이 열 가지가 넘어가니까 스트레스야.,건강할 때 관리 좀 잘할걸 하는 생각이 들더라고.,약을 잘 챙겨 먹고 나을 수 있도록 노력해야지.,NaN
2,1,고등학교에 올라오니 중학교 때보다 수업이 갑자기 어려워져서 당황스러워.,아직 학기 초인데 내가 수업에 잘 따라갈 수 있을지 걱정돼.,중학교 수업을 들을 때보다 훨씬 더 열심히 할 거야.,선생님이 강조하신 부분을 필기하고 집에서 매일 수업 내용을 복습하려고 해.
3,7,재취업이 돼서 받게 된 첫 월급으로 온 가족이 외식을 할 예정이야. 너무 행복해.,퇴직 후 다시는 돈을 못 벌 줄 알았는데 이렇게 월급으로 가족에게 맛있는 밥을 살 ...,회사생활을 열심히 해서 계속 월급을 받을거야!,NaN
4,7,빚을 드디어 다 갚게 되어서 이제야 안도감이 들어.,빚도 다 갚았으니 당분간은 아무 생각도 안 하며 살고 싶어.,NaN,NaN
...,...,...,...,...,...
46004,7,부동산 임대 소득으로 현재 여유롭게 살 수 있어서 좋단다.,그래서 마음의 안정을 가질 수 있었던 것 같아.,돈을 함부로 쓰지 말고 지금 현재에 만족하면서 지내야지.,NaN
46005,8,폐결핵은 이미 완치된 것 같은데 약을 한 달이나 더 먹으라고 하네? 아직 안 나은 ...,이대로 혹시 더 심해져서 죽게 되는 건 아닐까?,의사 선생님에게 악화될 확률은 얼마나 되는지 정확히 물어봐야겠어.,NaN
46006,5,연애하고 싶은데 소개팅만 나가면 꽝이야. 이러다가 난 결혼 못 하고 늙어 죽을 거야.,내가 게을러서 다른 방법을 아직 생각하진 못했어. 댄스 동호회는 좀 괜찮은 것 같은...,NaN,NaN
46007,7,은행 대출이 막혀서 생활비를 구할 수가 없어. 이제 어떻게 살아야 하나 막막해.,당장 내일 밥은 어떡하고 내 병원비는 어쩌지.,어떻게 해야 할 지 모르겠어. 눈 앞이 캄캄해.,NaN


In [ ]:
nullcheck1 = chat_data['sentence1'].isnull()
nullcheck2 = chat_data['sentence2'].isnull()
nullcheck3 = chat_data['sentence3'].isnull()
nullcheck4 = chat_data['sentence3.1'].isnull()

In [ ]:
data_list = []
for check1,check2, check3, check4, label, sentence1, sentence2, sentence3, sentence4 in zip(nullcheck1,nullcheck2, nullcheck3, nullcheck4, chat_data['situation'], chat_data['sentence1'], chat_data['sentence2'], chat_data['sentence3'], chat_data['sentence3.1'])  :
    data = []
    sentence = ""

    if check1 != True :
      sentence = sentence1

    if check2 != True :
      sentence = sentence + " " + sentence2

    if check3 != True :
      sentence = sentence + " " + sentence3
    
    if check4 != True :
      sentence = sentence + " " + sentence4
    
    
    data.append(sentence)
    data.append(str(label))

    data_list.append(data)

In [ ]:
data_list[0]

['아내가 드디어 출산하게 되어서 정말 신이 나. 아 지금 정말 신이 나. 아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지. ',
 '5']

In [ ]:
#가족관계 = 0
#학업 및 진로 = 1
#학교폭력/따돌림 = 2
#대인관계(부부, 자녀) = 3
#대인관계 = 4
#연애, 결혼, 출산 = 5
#진로, 취업, 직장 = 6
#재정, 은퇴, 노후준비 = 7
#건강, 죽음 = 8
#건강 = 9
# 직장, 업무 스트레스 = 10
#재정 = 11

가족관계 = 0

학업 및 진로 = 1

학교폭력/따돌림 = 2

대인관계 = 3

 직장, 업무 스트레스 = 4

연애, 결혼, 출산 = 5

진로, 취업, 직장 = 6

재정, 은퇴, 노후준비 = 7

건강, 죽음 = 8

In [ ]:
dataset_train, dataset_test = train_test_split(data_list, test_size=0.1, random_state=0)

In [ ]:
class BERTDataset(Dataset):
    def __init__(self, dataset, sent_idx, label_idx, bert_tokenizer, max_len,
                 pad, pair):
        transform = nlp.data.BERTSentenceTransform(bert_tokenizer, max_seq_length=max_len, pad=pad, pair=pair)

        self.sentences = [transform([i[sent_idx]]) for i in dataset]
        self.labels = [np.int32(i[label_idx]) for i in dataset]

    def __getitem__(self, i):
        return (self.sentences[i] + (self.labels[i], ))

    def __len__(self):
        return (len(self.labels))

In [ ]:
max_len = 200
batch_size = 32
warmup_ratio = 0.1
num_epochs = 10
max_grad_norm = 1
log_interval = 200
learning_rate =  4e-5

In [ ]:
print(len(dataset_train))
print(len(dataset_test))

41408
4601


In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)

data_train = BERTDataset(dataset_train, 0, 1, tok, max_len, True, False)
data_test = BERTDataset(dataset_test, 0, 1, tok, max_len, True, False)

using cached model


In [ ]:
data_train[0]

(array([   2, 3647, 5400,  806, 2110, 3818, 7318,  633, 4627, 6897, 6553,
         806, 1574, 5760, 3343, 7819, 1958, 6005, 2123, 6116, 2229, 6855,
         517,   54, 3672, 4746, 5760, 1861, 4468, 7096, 6844,  517,   54,
           3,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1,    1,    1,    1,    1,    1,    1,
           1,    1,    1,    1,    1, 

In [ ]:
train_dataloader = torch.utils.data.DataLoader(data_train, batch_size=batch_size)
test_dataloader = torch.utils.data.DataLoader(data_test, batch_size=batch_size)

In [ ]:
class BERTClassifier(nn.Module):
    def __init__(self,
                 bert,
                 hidden_size = 768,
                 num_classes=9,   ##클래스 수 조정##
                 dr_rate=None,
                 params=None):
        super(BERTClassifier, self).__init__()
        self.bert = bert
        self.dr_rate = dr_rate
                 
        self.classifier = nn.Linear(hidden_size , num_classes)
        if dr_rate:
            self.dropout = nn.Dropout(p=dr_rate)
    
    def gen_attention_mask(self, token_ids, valid_length):
        attention_mask = torch.zeros_like(token_ids)
        for i, v in enumerate(valid_length):
            attention_mask[i][:v] = 1
        return attention_mask.float()

    def forward(self, token_ids, valid_length, segment_ids):
        attention_mask = self.gen_attention_mask(token_ids, valid_length)
        
        _, pooler = self.bert(input_ids = token_ids, token_type_ids = segment_ids.long(), attention_mask = attention_mask.float().to(token_ids.device), return_dict=False)
        if self.dr_rate:
            out = self.dropout(pooler)
        return self.classifier(out)

In [ ]:
#BERT 모델 불러오기
model = BERTClassifier(bertmodel,  dr_rate=0.5).to(device)

#optimizer와 schedule 설정
no_decay = ['bias', 'LayerNorm.weight']
optimizer_grouped_parameters = [
    {'params': [p for n, p in model.named_parameters() if not any(nd in n for nd in no_decay)], 'weight_decay': 0.005},
    {'params': [p for n, p in model.named_parameters() if any(nd in n for nd in no_decay)], 'weight_decay': 0.0}
]

optimizer = AdamW(optimizer_grouped_parameters, lr=learning_rate)
loss_fn = nn.CrossEntropyLoss()

t_total = len(train_dataloader) * num_epochs
warmup_step = int(t_total * warmup_ratio)

scheduler = get_cosine_schedule_with_warmup(optimizer, num_warmup_steps=warmup_step, num_training_steps=t_total)

#정확도 측정을 위한 함수 정의
def calc_accuracy(X,Y):
    max_vals, max_indices = torch.max(X, 1)
    train_acc = (max_indices == Y).sum().data.cpu().numpy()/max_indices.size()[0]
    return train_acc
    

In [ ]:
# for data in train_dataloader :
#     print(data)
#     break

In [ ]:
for e in range(num_epochs):
    train_acc = 0.0
    test_acc = 0.0
    model.train()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(train_dataloader)):
        optimizer.zero_grad()
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        loss = loss_fn(out, label)
        loss.backward()
        torch.nn.utils.clip_grad_norm_(model.parameters(), max_grad_norm)
        optimizer.step()
        scheduler.step()  # Update learning rate schedule
        train_acc += calc_accuracy(out, label)
        if batch_id % log_interval == 0:
            print("epoch {} batch id {} loss {} train acc {}".format(e+1, batch_id+1, loss.data.cpu().numpy(), train_acc / (batch_id+1)))
    print("epoch {} train acc {}".format(e+1, train_acc / (batch_id+1)))
    
    model.eval()
    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(notebook.tqdm(test_dataloader)):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)
        valid_length= valid_length
        label = label.long().to(device)
        out = model(token_ids, valid_length, segment_ids)
        test_acc += calc_accuracy(out, label)
    print("epoch {} test acc {}".format(e+1, test_acc / (batch_id+1)))

  0%|          | 0/1294 [00:00<?, ?it/s]

epoch 1 batch id 1 loss 2.246034860610962 train acc 0.0625
epoch 1 batch id 201 loss 1.8649746179580688 train acc 0.2035136815920398
epoch 1 batch id 401 loss 1.1883034706115723 train acc 0.3626870324189526
epoch 1 batch id 601 loss 0.8589463829994202 train acc 0.46698211314475874
epoch 1 batch id 801 loss 0.6504119634628296 train acc 0.530391697877653
epoch 1 batch id 1001 loss 0.6710875630378723 train acc 0.5696491008991009
epoch 1 batch id 1201 loss 0.8372682332992554 train acc 0.5973407577019151
epoch 1 train acc 0.6065736089644513


  0%|          | 0/144 [00:00<?, ?it/s]

epoch 1 test acc 0.7423177083333333


  0%|          | 0/1294 [00:00<?, ?it/s]

...

epoch 9 test acc 0.7863715277777777


  0%|          | 0/1294 [00:00<?, ?it/s]

epoch 10 batch id 1 loss 0.002859278814867139 train acc 1.0
epoch 10 batch id 201 loss 0.13429103791713715 train acc 0.9805659203980099
epoch 10 batch id 401 loss 0.004746966063976288 train acc 0.9829332917705735
epoch 10 batch id 601 loss 0.17625927925109863 train acc 0.9841410149750416
epoch 10 batch id 801 loss 0.004102678038179874 train acc 0.9845115480649188
epoch 10 batch id 1001 loss 0.0505780428647995 train acc 0.9849213286713286
epoch 10 batch id 1201 loss 0.007023255806416273 train acc 0.9846482098251457
epoch 10 train acc 0.9850270479134466


  0%|          | 0/144 [00:00<?, ?it/s]

epoch 10 test acc 0.7844184027777777


In [ ]:
torch.save(model, "situation7.pth")
#situation1 은 train 89% test 73%
#situation2 은 train 96.5% test 72.8%
#situation3 은  99%, 73% epoch 9
#situaation4도  99%  73% epoch 4
#situaation5도  99%  73% epoch 4
#situaation6 class 9개 89% 77%
#situation7은 99% 78% epoch 9

In [ ]:
#토큰화
tokenizer = get_tokenizer()
tok = nlp.data.BERTSPTokenizer(tokenizer, vocab, lower=False)


def predict(predict_sentence):

    data = [predict_sentence, '0']
    dataset_another = [data]

    another_test = BERTDataset(dataset_another, 0, 1, tok, max_len, True, False)
    test_dataloader = torch.utils.data.DataLoader(another_test, batch_size=batch_size)
    
    model.eval()

    for batch_id, (token_ids, valid_length, segment_ids, label) in enumerate(test_dataloader):
        token_ids = token_ids.long().to(device)
        segment_ids = segment_ids.long().to(device)

        valid_length= valid_length
        label = label.long().to(device)

        out = model(token_ids, valid_length, segment_ids)


        test_eval=[]
        for i in out:
            logits=i
            logits = logits.detach().cpu().numpy()

            if np.argmax(logits) == 0:
                test_eval.append("가족관계")
            elif np.argmax(logits) == 1:
                test_eval.append("학업 및 진로")
            elif np.argmax(logits) == 2:
                test_eval.append("학교폭력/따돌림")
            elif np.argmax(logits) == 3:
                test_eval.append("대인관계(부부, 자녀)")
            elif np.argmax(logits) == 4:
                test_eval.append("대인관계")
            elif np.argmax(logits) == 5:
                test_eval.append("연애, 결혼, 출산")
            elif np.argmax(logits) == 6:
                test_eval.append("진로, 취업, 직장")
            elif np.argmax(logits) == 7:
                test_eval.append("재정, 은퇴, 노후준비")
            elif np.argmax(logits) == 8:
                test_eval.append("건강, 죽음")
            elif np.argmax(logits) == 9:
                test_eval.append("건강")
            elif np.argmax(logits) == 10:
                test_eval.append("직장, 업무 스트레스")
            elif np.argmax(logits) == 11:
                test_eval.append("재정")

            print(logits)
        print(">> 입력하신 내용에서 " + test_eval[0] + " 상황입니다.")

using cached model


In [ ]:
#질문 무한반복하기! 0 입력시 종료

end = 1
while end == 1 :
    sentence = input("하고싶은 말을 입력해주세요 : ")
    if sentence == 0 :
        break
    predict(sentence)
    print("\n")

하고싶은 말을 입력해주세요 : 돈버는 거 정말 귀찮아
[ 0.15945753 -0.82328266 -1.1253926  -0.15447387  1.0187023  -0.7059809
 -0.26569748  1.3082827  -0.500366   -1.3852215  -1.6688879   5.295386  ]
>> 입력하신 내용에서 재정 상황입니다.


하고싶은 말을 입력해주세요 : 아내가 드디어 출산하게 되어서 정말 신이 나. 아 지금 정말 신이 나. 아기가 점점 클게 벌써 기대가 되네. 내가 많이 놀아줘야지.
[-0.9397611  -0.8115472  -0.82847023  0.90904164  0.5413477   7.344616
 -0.6979294  -0.4296977  -1.7551643  -0.26204985 -1.2668182  -1.3639736 ]
>> 입력하신 내용에서 연애, 결혼, 출산 상황입니다.




KeyboardInterrupt: ignored